In [ ]:
''' 
import numpy as np
import pandas as pd
    
from dotenv import load_dotenv
import os
    
import unicodedata
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, f1_score, fbeta_score, roc_curve, average_precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import ConfusionMatrixDisplay
import logging
import json
import re
import string 
import joblib
import warnings
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import mlflow
import mlflow.sklearn
from sklearn.pipeline import Pipeline
from mlflow.models.signature import infer_signature
from datetime import datetime
'''
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.metrics import precision_recall_curve, auc, precision_score, recall_score, f1_score, fbeta_score, roc_curve, average_precision_score
from nltk.corpus import stopwords
import nltk
import logging
import mlflow

from NB_funciones import CargarDatos
from NB_funciones import run_ml_experiment
from NB_funciones import get_champion_model
from NB_funciones import train_challenger_model
from NB_funciones import register_challenger_model
from NB_funciones import compare_models
from NB_funciones import log_info, log_error

#warnings.filterwarnings('ignore')

In [3]:
nltk.download('stopwords')
nltk.download('punkt_tab')
english_stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to /home/alejo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/alejo/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [5]:
# Configuración del logging
logging.basicConfig(
    filename="errores_entrenamiento.log",
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

In [ ]:
# Funciones para registrar mensajes tanto en log como en consola
#def log_info(message):
#    print(message)
#    logging.info(message)

#def log_error(message):
#    print(f'ERROR: {message}')
#    logging.error(message)

In [6]:
ruta = CargarDatos("customer_support_twitter_twcs")  # La clave en .env debe ser RUTA_ARCHIVO
data_cst_twcs = ruta.cargar_csv()
print(data_cst_twcs.shape)

INFO:root:Archivo /home/alejo/proyectos/MLE_Nequi/datasets/Customer_Support_Twitter/twcs/twcs.csv cargado correctamente.


(2811774, 7)


In [5]:
data_cst_twcs['inbound'] = data_cst_twcs['inbound'].astype('int')

In [10]:
try:
    X_train, X_test, y_train, y_test = train_test_split(data_cst_twcs['text'], data_cst_twcs['inbound'], 
                                                        test_size=0.3, stratify=data_cst_twcs['inbound'], 
                                                        random_state=42)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, 
                                                        test_size=0.3, stratify=y_train, 
                                                        random_state=42)
    print('X_train: ', X_train.shape)
    print('X_valid: ', X_valid.shape)
    print('X_test: ', X_test.shape)
    print('y_train: ', y_train.shape)
    print('y_valid: ', y_valid.shape)
    print('y_test: ', y_test.shape)
    log_info('Datos divididos en conjuntos de entrenamiento, validación y prueba correctamente.')
except Exception as e:
    log_error(f'Error al dividir los datos: {e}')
    raise e

INFO:root:Datos divididos en conjuntos de entrenamiento, validación y prueba correctamente.


X_train:  (1377768,)
X_valid:  (590473,)
X_test:  (843533,)
y_train:  (1377768,)
y_valid:  (590473,)
y_test:  (843533,)
Datos divididos en conjuntos de entrenamiento, validación y prueba correctamente.


In [15]:
# Set the experiment name
try:
    mlflow.create_experiment("experimento_nuevo")
    print("Experimento creado")
    logging.info("Experiment created successfully.")
except:
    mlflow.set_experiment("experimento_nuevo")
    print("Experimento ya existe")
    logging.info("Experiment already exists, set to existing experiment.")

INFO:root:Experiment created successfully.


Experimento creado


In [16]:
run_ml_experiment(X_train, y_train, X_test, y_test, english_stopwords, model_name="modelo_nuevo")

Iniciando el experimento...
0:	learn: 0.6316539	total: 327ms	remaining: 2m 43s
100:	learn: 0.3638261	total: 18.3s	remaining: 1m 12s
200:	learn: 0.3455131	total: 36.4s	remaining: 54.2s
300:	learn: 0.3375544	total: 54.3s	remaining: 35.9s
400:	learn: 0.3330456	total: 1m 16s	remaining: 18.8s
499:	learn: 0.3297846	total: 1m 32s	remaining: 0us


Successfully registered model 'modelo_nuevo'.
Created version '1' of model 'modelo_nuevo'.
INFO:root:Modelo registrado con precisión: 0.8513466574514571


Modelo registrado con precisión: 0.8513466574514571


In [ ]:
get_champion_model(model_name="modelo_nuevo") # experiment_name="experimento_nuevo"

INFO:root:No se encontró un modelo modelo_nuevo en producción. Buscando en staging...
INFO:root:No se encontró un modelo modelo_nuevo en staging. Buscando la versión más reciente...
INFO:root:Modelo campeón cargado: modelo_nuevo version 1


(Pipeline(steps=[('tfidf',
                  TfidfVectorizer(max_features=100,
                                  stop_words=['a', 'about', 'above', 'after',
                                              'again', 'against', 'ain', 'all',
                                              'am', 'an', 'and', 'any', 'are',
                                              'aren', "aren't", 'as', 'at', 'be',
                                              'because', 'been', 'before',
                                              'being', 'below', 'between',
                                              'both', 'but', 'by', 'can',
                                              'couldn', "couldn't", ...],
                                  token_pattern='\\b\\w+\\b')),
                 ('catboost',
                  <catboost.core.CatBoostClassifier object at 0x7f19c8a4eb10>)]),
 'c893536222134b59a599033076b3335c')

In [23]:
# Entrenar el modelo challenger
challenger_model = train_challenger_model(X_train = X_valid, y_train = y_valid)

INFO:root:Entrenando modelo desafiante con parámetros: {'iterations': 500, 'depth': 6, 'learning_rate': 0.1, 'verbose': 100}


0:	learn: 0.6316481	total: 138ms	remaining: 1m 8s
100:	learn: 0.3679895	total: 7.32s	remaining: 28.9s
200:	learn: 0.3489835	total: 13.9s	remaining: 20.8s
300:	learn: 0.3405854	total: 21.4s	remaining: 14.1s
400:	learn: 0.3356854	total: 28.2s	remaining: 6.95s


INFO:root:Modelo desafiante entrenado correctamente


499:	learn: 0.3323711	total: 35.3s	remaining: 0us


In [24]:
# Definir métricas de rendimiento (ejemplo)
metrics = {
    "accuracy": accuracy_score(y_test, challenger_model.predict(X_test)),
    "f1_score": f1_score(y_test, challenger_model.predict(X_test), average='weighted')
}

In [25]:
# Registrar modelo en MLflow
registro_exitoso = register_challenger_model(challenger_model, metrics, X_train, y_train, is_champion=False)

if registro_exitoso:
    log_info("El modelo challenger ha sido registrado exitosamente.")
else:
    log_error("Hubo un problema al registrar el modelo challenger.")


Successfully registered model 'Modelo_Campeon_challenger'.
Created version '1' of model 'Modelo_Campeon_challenger'.
INFO:root:El modelo challenger ha sido registrado exitosamente.


El modelo challenger ha sido registrado exitosamente.


In [26]:
# Definir el nombre del modelo que registraste en MLflow
model_name = "modelo_nuevo"

# Cargar el modelo desde MLflow
loaded_model = mlflow.sklearn.load_model(f"models:/{model_name}/latest")

# Verificar que el modelo se cargó correctamente
print("Modelo cargado:", loaded_model)


Modelo cargado: Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=100,
                                 stop_words=['a', 'about', 'above', 'after',
                                             'again', 'against', 'ain', 'all',
                                             'am', 'an', 'and', 'any', 'are',
                                             'aren', "aren't", 'as', 'at', 'be',
                                             'because', 'been', 'before',
                                             'being', 'below', 'between',
                                             'both', 'but', 'by', 'can',
                                             'couldn', "couldn't", ...],
                                 token_pattern='\\b\\w+\\b')),
                ('catboost',
                 <catboost.core.CatBoostClassifier object at 0x7f19c2b2eba0>)])


In [29]:
champion_metrics = {
    "accuracy": accuracy_score(y_test, loaded_model.predict(X_test)),
    "f1_score": f1_score(y_test, loaded_model.predict(X_test), average='weighted')
}

In [30]:
compare_models(champion_metrics = champion_metrics, challenger_metrics = metrics, primary_metric="accuracy", threshold=0.5)

INFO:root:El modelo desafiante NO supera al campeón en accuracy: 0.8500 vs 0.8513
INFO:root:Diferencia absoluta: -0.0014, Diferencia porcentual: -0.16%


(False,
 {'accuracy': {'champion': 0.8513466574514571,
   'challenger': 0.8499620050430747,
   'absolute_diff': -0.0013846524083823697,
   'percent_diff': -0.16264260818588122},
  'f1_score': {'champion': 0.8498914196426405,
   'challenger': 0.8486554533144898,
   'absolute_diff': -0.001235966328150706,
   'percent_diff': -0.145426380309899}},
 {'champion': 0.8513466574514571,
  'challenger': 0.8499620050430747,
  'absolute_diff': -0.0013846524083823697,
  'percent_diff': -0.16264260818588122})